*Based on https://www.cse.unr.edu/~bebis/CS485/*

# Edge Detections
## Definition of Edges
    Edges are significant local changes of intensity in an image.

## What Causes Intensity Changes?
- Geometric events
    - surface orientation (boundary) discontinuities
    - depth discontinuities
    - color and texture discontinuities
- Non-Geometric events
    - illumination changes
    - specularities
    - shadows
    - inter-reflections

![title](img/intensity-change.png)

## Goal of Edge Detection
    Produce a line drawing of a scene from an image of that scene.
    Important features can be extracted from the edges of an image (e.g., corners, lines, curves)
    These features are used by higher-level comptuer vision algorithms (e.g., recognition)

### Effect of Illumination
![title](img/illumination.png)

## Edge Descriptors
- Edge Direction
    - perpendicular to the direction of maximum intensity change (i.e., edge normal)
- Edge Strength
    - related to the local image contrast along the normal.
- Edge Position
    - the image position at which the edge is located.

## Modeling Intensity Changes
- Step edge
    - the image intensity abruptly changes from one value on one side of the discontinuity to a different value on the opposite side.
    
![title](img/step-edge.png)
- Ramp edge
    - a step edge where the intensity change is not instantaneous but occur over a finite distance.

![title](img/ramp-edge.png)
- Ridge edge
    - the image intensity abruptly changes value but then returns to the starting value within some short distance (i.e., usually generated by lines.)

![title](img/ridge-edge.png)
## Main Steps in Edge Detection
- Smoothing
    - supress as much noise as possible, without destroying true edges

- Enhancement
    - apply differentiation to enhance the quality of edges (i.e., sharpening)

- Thresholding
    - determine which edge pixels should be discarded as noise and which should be retained (i.e., threshold edge magnitude)

- Localization
    - determine the exact edge location

sub-pixel resoultion might be required for some applications to estimate the location of an edge to better than the spacing between pixels.

## Edge Detection Using Derivatives
    Often, points that lie on an edge are detected by,
1. Detecting the local maxima or minima of the first derivative.
2. Detecting the zero-crossings of the second derivative.
![title](img/edge_using_derivatives.png)

### Image Derivatives
- Option 1
    - reconstruct a continuous image $f(x,y)$ then compute the derivative.
- Option 2
    - take discrete derivative (i.e., finite differences)
![title](img/edge-first-derivates.png)

### Edge Detection Using Second Derivative
- Approximate finding maxima/minima of gradient magnitude by finding places where:
$$
\frac{df^2}{dx^2} = 0
$$
- Can't always find discrete pixels where the second drivative is zero - look for **zero-crossing** instead.
- 1D Functions:
$$
f''(x) = \lim_{h→0}{\frac{f'(x+h)-f'(x)}{h}} ≈ f'(x+1)-f'(x) = f(x+2)-2f(x+1)+f(x) \newline
(h=1), (\text{centered at } x+1)
$$
$$
\text{Replace }x+1\text{ with }x\text{ } (\text{i.e., }\text{centered at } x) \newline
f''(x) ≈ f(x+1)-2f(x)+f(x-1)
$$
$$
\text{mask : }\begin{bmatrix}1 & -2 & 1\end{bmatrix}
$$
![title](img/edge-second-derivative.png)


- Four cases of zero-crossings: \
    $\{+, -\}, \{+,0,-\}, \{-,+\}, \{-,0,+\}$
- Slope of zero-crossing \
    $\{a, -b\} = |a+b|$
- To detect "strong" zero-crossing, threshold the slope.

### Effect Smoothing on Derivates
![title](img/smoothing-derivatives-fd-.png)
![title](img/smoothing-derivatives.png)
![title](img/combine-smoothing-derivatives.png)

### Mathematical Interpretation of combining smoothing with differentiation
- Solution does not exist or it is not unique or it does not depend continuously on initial data
    - Regularization : Impose additional constraints
- Smoothing performs image interpolation

### Edge Detection Using First Derivative (Gradient)
- 2D Functions
    - The first derivate of an image can be computed using the gradient
    $$
    \nabla{f} = grad(f) = \begin{pmatrix}\frac{\partial{f}}{\partial{x}} \\ \frac{\partial{f}}{\partial{y}}\end{pmatrix}
    $$
- The gradient is a vector which has magnitude and direction:
    $$
    magnitude(grad(f)) = \sqrt{\frac{\partial{f}^2}{\partial{x}} + \frac{\partial{f}^2}{\partial{y}}} \newline
    direction(grad(f)) = \tan^{-1}(\frac{\partial{f}}{\partial{y}}/\frac{\partial{f}}{\partial{x}}) \newline
    $$
                                                or
    $$
    |\frac{\partial{f}}{\partial{x}} + \frac{\partial{f}}{\partial{y}}|\text{ (approximation)}
    $$

- Magnitude: indicates edge strength.
- Direction: indicates edge direction. \
![title](img/edge-direction.png)
![title](img/approximate-gradient.png)

- Cartesian vs pixel-coordinates:
    - i corresponds to x direction
    - i to -y direction
![title](img/cartesian-vs-pixel.png)
![title](img/approximate-gradient-edge.png)

- We can implement $\frac{\partial{f}}{\partial{x}}$ and $\frac{\partial{f}}{\partial{y}}$ using the following masks
![title](img/approximate-gradient-implement.png)

- A different approximation of the gradient:
![title](img/approximate-gradient-implement-center.png)
$\frac{\partial{f}}{\partial{x}}$ and $\frac{\partial{f}}{\partial{y}}$ can be implemented using the following masks:
![title](img/approximate-gradient-implement-center-mask.png)


### Another Approximation
- Consider the arrangement of pixels about the pixel (i, j):
    3 x 3 neighborhood : \
![title](img/snail-approximation.png)

- The partial derivatives $\frac{\partial{f}}{\partial{x}}$. $\frac{\partial{f}}{\partial{y}}$ can be computed by:

$$
M_x = (a_2+ca_3+a_4) - (a_0+ca_7+a_6) \newline
M_y = (a_6+ca_5+a_4) - (a_0+ca_1+a_2)
$$

- The constant $c$ implies the emphasis given to pixels closer to the center of the mask

#### Prewitt Operator
- Setting c = 1, we get the previtt operator \
![title](img/prewitt.png)
![title](img/prewitt-example.png)

#### Sobel Operator
- Setting c = 2, we get the Sobel operator \
![title](img/sobel.png)

#### Edge Detection Steps Using Gradient
1. Smooth the input image $\hat{f}(x, y) = f(x, y) * G(x, y)$
2. $\hat{f_x} = \hat{f}(x, y) * M_x(x, y) → \frac{\partial{f}}{\partial{x}}$
3. $\hat{f_y} = \hat{f}(x, y) * M_x(x, y) → \frac{\partial{f}}{\partial{y}}$
4. $magn(x,y) = |\hat{f_x}| + |\hat{f_y}| \text{ (i.e., sqrt is costly!)}$
5. $dir(x,y) = tan^{-1}(\hat{f_y} / \hat{f_x})$
6. If $magn(x,y) > T, $ then possible edge point

#### Isotropic property of gradient magnitude
- The magnitude of the gradient detects edges in all directions
$$
\nabla = \sqrt{(\frac{d}{dx}I)^{2} + (\frac{d}{dy}I})^{2}
$$

## Practical Issues
- Noise-suppression-localization tradeoff.
    - Smoothing depends on mask size (e.g., depends on $\sigma$ for gaussian filters)
    - Larger mask sizes reduce noise, but worsen localization (i.e., add uncertainty to the location of the edge) and vice versa.
![title](img/threshold.png)

- Edge thinning and linking
![title](img/edge-line.png)

## Criteria for Optimal Edge Detection
- Good detection
    - Minimize the probability of false positives (i.e., spurious edges).
    - Minimize the probability of false negatives (i.e., missing real edges).

- Good localization
    - Detected edges must be as close as possible tot the true edges

- Single response
    - Minimize the number of local maxima around teh true edge.

## Canny Edge Detector
    Canny has shown that the first derivative of the Gaussian closely approximates the operator that optimizes the product of singal-to-noise ratio and localization

![title](img/canny-edge.png)
